In [19]:
import requests
import regex as re
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import PyPDF2
from io import BytesIO
from bs4 import BeautifulSoup


In [62]:
from danlp.models import load_bert_base_model
model = load_bert_base_model()
vecs_embedding, sentence_embedding, tokenized_text =model.embed_text('Han sælger frugt')

KeyboardInterrupt: 

# This tells me that I definitely need to do some preprocessing on the data. Raw embeddings are not going to work.

In [74]:
import re
from danlp.models import load_bert_base_model
from tqdm import tqdm
import random

# Load the BERT model
model = load_bert_base_model()


# Function to embed questions using the BERT model
def embed_questions(questions):
    embeddings = {}
    #take a random subset of 10000 questions
    
    random_keys = random.sample(list(questions.keys()), 10000)
    random_subset = {k: questions[k] for k in random_keys}


    for question_id, question_text in tqdm(random_subset.items(), desc="Embedding questions", total=min(10000, len(random_subset))):
        _, sentence_embedding, _ = model.embed_text(question_text)
        embeddings[question_id] = sentence_embedding
    return embeddings

# Embed the first 10,000 filtered questions
question_embeddings = embed_questions(question_dict)


Some weights of the model checkpoint at C:\Users\rune7\.danlp\bert.botxo.pytorch were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Embedding questions: 100%|██████████| 10000/10000 [10:05<00:00, 16.52it/s]


In [75]:
import numpy as np
from scipy.spatial.distance import cdist

def embed_user_question(question):
    _, user_embedding, _ = model.embed_text(question)
    return user_embedding

def find_nearest_k_questions(user_embedding, question_embeddings, k=5):
    user_embedding = user_embedding.numpy()  # Convert user_embedding to a NumPy array
    question_ids = np.array(list(question_embeddings.keys()))
    
    # Convert embeddings to NumPy arrays and stack them along a new dimension
    embeddings = np.stack([emb.numpy() for emb in question_embeddings.values()])
    print("User embedding shape:", user_embedding.shape)
    print("Embeddings shape:", embeddings.shape)
    similarities = 1 - cdist(user_embedding.reshape(1, -1), embeddings, metric='cosine').reshape(-1)


    # Find the nearest k question ids
    nearest_indices = np.argpartition(similarities, -k)[-k:]
    nearest_question_ids = question_ids[nearest_indices]
    nearest_similarities = similarities[nearest_indices]

    # Sort the nearest questions by similarity
    sorted_indices = np.argsort(-nearest_similarities)
    nearest_question_ids = nearest_question_ids[sorted_indices]
    nearest_similarities = nearest_similarities[sorted_indices]

    return nearest_question_ids, nearest_similarities

# User question
user_question = "Hvad har vi gjort for at beskytte udsatte naturområder ved Lillebælt?"

# Embed the user question
user_embedding = embed_user_question(user_question)




# Find the nearest k questions
k = 5
nearest_question_ids, nearest_similarities = find_nearest_k_questions(user_embedding, question_embeddings, k)

# Print the nearest questions and their similarities
for i, (question_id, similarity) in enumerate(zip(nearest_question_ids, nearest_similarities), 1):
    print(f"{i}. Question ID: {question_id}, Similarity: {similarity:.4f}, Text: {question_dict[question_id]}")


User embedding shape: (768,)
Embeddings shape: (10000, 768)
1. Question ID: 9379, Similarity: 0.6511, Text:  Hvad vil ministeren gøre for at motivere private virksomheder til at styrke deres it-sikkerhed?
2. Question ID: 738623, Similarity: 0.6375, Text:  Støtter regeringen Socialdemokratiets forslag om at etablere flere stenrev i kystnære områder til gavn for fiskebestanden?
3. Question ID: 899990, Similarity: 0.6180, Text:  Anser ministeren statens prioritering af udsatte og sårbare borgere med udviklingshandicap for forsvarlig? 
4. Question ID: 4392, Similarity: 0.6103, Text:  Hvad er ministerens holdning til træer langs vejene med hensyn til arbejde med at sikre vejenes funktion?
5. Question ID: 712722, Similarity: 0.6049, Text:  Hvad har ministeren tænkt sig at gøre over for den hærværk, der rammer lokale erhvervsdrivende på Nørrebro? 


In [2]:
import sys
#append the models folder path to the system path
sys.path.append('C:/Users/Rune/Documents/GitHub/knowledge-graph/src/features')
from bert_base import BertModel

ModuleNotFoundError: No module named 'bert_base'

In [ ]:



def embed_user_question(question):
    _, user_embedding, _ = model.embed_text(question)
    return user_embedding

def find_nearest_k_questions(user_embedding, question_embeddings, k=5):
    user_embedding = user_embedding.numpy()  # Convert user_embedding to a NumPy array
    question_ids = np.array(list(question_embeddings.keys()))
    
    # Convert embeddings to NumPy arrays and stack them along a new dimension
    embeddings = np.stack([emb.numpy() for emb in question_embeddings.values()])
    print("User embedding shape:", user_embedding.shape)
    print("Embeddings shape:", embeddings.shape)
    similarities = 1 - cdist(user_embedding.reshape(1, -1), embeddings, metric='cosine').reshape(-1)


    # Find the nearest k question ids
    nearest_indices = np.argpartition(similarities, -k)[-k:]
    nearest_question_ids = question_ids[nearest_indices]
    nearest_similarities = similarities[nearest_indices]

    # Sort the nearest questions by similarity
    sorted_indices = np.argsort(-nearest_similarities)
    nearest_question_ids = nearest_question_ids[sorted_indices]
    nearest_similarities = nearest_similarities[sorted_indices]

    return nearest_question_ids, nearest_similarities

# User question
user_question = "Hvad har vi gjort for at beskytte udsatte naturområder ved Lillebælt?"

# Embed the user question
user_embedding = embed_user_question(user_question)




# Find the nearest k questions
k = 5
nearest_question_ids, nearest_similarities = find_nearest_k_questions(user_embedding, question_embeddings, k)

# Print the nearest questions and their similarities
for i, (question_id, similarity) in enumerate(zip(nearest_question_ids, nearest_similarities), 1):
    print(f"{i}. Question ID: {question_id}, Similarity: {similarity:.4f}, Text: {question_dict[question_id]}")
